In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 48 kB/s 
     |████████████████████████████████| 198 kB 82.3 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805911 sha256=816516a48135944675b91edf6e97c7946f00cddf5adc40fde37eea2b76d36829
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Linear Regression").getOrCreate()

In [4]:
spark

In [10]:
df = spark.read.csv("/content/drive/MyDrive/Pyspark/tips.csv", header=True, inferSchema=True)
df.show()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|
|     15.42|1.57|  Male|    No|Sun|Dinner|   2|
|     18.43| 3.0|  Male|    No|Sun|Dinner|   4|
|     14.83|3.02|Female|    No|Sun|Dinner|   2|
|     21.58|3.92|  Male|    No|Sun|Dinner|   2|
|     10.33|1.67|Female|    No|Sun|Dinner|   3|
|     16.29|3.71|  Male|    No|Sun|Dinne

In [15]:
df.printSchema()

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [14]:
df.describe()

DataFrame[summary: string, total_bill: string, tip: string, sex: string, smoker: string, day: string, time: string, size: string]

Encoding the categorical column

In [24]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCols=["sex","smoker","day","time"], outputCols=["Sex_indexed","smoker_index","day_index","time_index"])
df_1 = indexer.fit(df).transform(df)

In [27]:
df_1.show(5)

+----------+----+------+------+---+------+----+-----------+------------+---------+----------+
|total_bill| tip|   sex|smoker|day|  time|size|Sex_indexed|smoker_index|day_index|time_index|
+----------+----+------+------+---+------+----+-----------+------------+---------+----------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|         0.0|      1.0|       0.0|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|         0.0|      1.0|       0.0|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|         0.0|      1.0|       0.0|
+----------+----+------+------+---+------+----+-----------+------------+---------+----------+
only showing top 5 rows



Lets create a dependent and independent variable

In [43]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["total_bill","size","Sex_indexed", "smoker_index","day_index","time_index"], outputCol="Independent_features")
assembler = assembler.transform(df_1)

In [44]:
assembler.show()

+----------+----+------+------+---+------+----+-----------+------------+---------+----------+--------------------+
|total_bill| tip|   sex|smoker|day|  time|size|Sex_indexed|smoker_index|day_index|time_index|Independent_features|
+----------+----+------+------+---+------+----+-----------+------------+---------+----------+--------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.0|         0.0|      1.0|       0.0|[16.99,2.0,1.0,0....|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|[10.34,3.0,0.0,0....|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|        0.0|         0.0|      1.0|       0.0|[21.01,3.0,0.0,0....|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|        0.0|         0.0|      1.0|       0.0|[23.68,2.0,0.0,0....|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|        1.0|         0.0|      1.0|       0.0|[24.59,4.0,1.0,0....|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|        0.0|         0.0|      1.

In [45]:
output = assembler.select("Independent_features","tip")
output.show()

+--------------------+----+
|Independent_features| tip|
+--------------------+----+
|[16.99,2.0,1.0,0....|1.01|
|[10.34,3.0,0.0,0....|1.66|
|[21.01,3.0,0.0,0....| 3.5|
|[23.68,2.0,0.0,0....|3.31|
|[24.59,4.0,1.0,0....|3.61|
|[25.29,4.0,0.0,0....|4.71|
|[8.77,2.0,0.0,0.0...| 2.0|
|[26.88,4.0,0.0,0....|3.12|
|[15.04,2.0,0.0,0....|1.96|
|[14.78,2.0,0.0,0....|3.23|
|[10.27,2.0,0.0,0....|1.71|
|[35.26,4.0,1.0,0....| 5.0|
|[15.42,2.0,0.0,0....|1.57|
|[18.43,4.0,0.0,0....| 3.0|
|[14.83,2.0,1.0,0....|3.02|
|[21.58,2.0,0.0,0....|3.92|
|[10.33,3.0,1.0,0....|1.67|
|[16.29,3.0,0.0,0....|3.71|
|[16.97,3.0,1.0,0....| 3.5|
|(6,[0,1],[20.65,3...|3.35|
+--------------------+----+
only showing top 20 rows



In [58]:
#Applying linear regression algorithms
from pyspark.ml.regression import LinearRegression

#Splitting the data for traing and testing data
train_data, test_data = output.randomSplit([0.75,0.25])

#Model building
regressor=LinearRegression(featuresCol="Independent_features", labelCol="tip")
regressor = regressor.fit(train_data)

In [66]:
predict = regressor.evaluate(test_data)
prediction

In [67]:
regressor.coefficients

DenseVector([0.0862, 0.2173, 0.0442, -0.1443, 0.0959, -0.0295])

In [68]:
regressor.intercept

0.6405724050878163

In [71]:
predict.predictions.show()

+--------------------+----+------------------+
|Independent_features| tip|        prediction|
+--------------------+----+------------------+
|(6,[0,1],[13.37,2...| 2.0| 2.227389365316781|
|(6,[0,1],[19.82,2...|3.18|2.7832130600197194|
|(6,[0,1],[20.08,3...|3.15| 3.022952200837088|
|(6,[0,1],[20.45,4...| 3.0|3.2721705054400885|
|(6,[0,1],[21.7,2.0])| 4.3| 2.945220586537785|
|[3.07,1.0,1.0,1.0...| 1.0|1.0224240778041598|
|[7.74,2.0,0.0,1.0...|1.44| 1.597974460938067|
|[8.77,2.0,0.0,0.0...| 2.0|1.9268573307966566|
|[9.6,2.0,1.0,1.0,...| 4.0| 1.898344914554264|
|[9.78,2.0,0.0,0.0...|1.73| 2.080259858870405|
|[11.17,2.0,1.0,0....| 1.5| 2.244259070705346|
|[11.24,2.0,0.0,1....|1.76| 1.899584217753615|
|[11.59,2.0,0.0,1....| 1.5|1.9297451934351697|
|[12.26,2.0,1.0,0....| 2.0| 2.338188966399331|
|[12.48,2.0,1.0,0....|2.52| 2.357147293970594|
|[12.66,2.0,0.0,0....| 2.5|2.2620750319430805|
|[13.81,2.0,0.0,0....| 2.0| 2.361175380611046|
|[13.94,2.0,0.0,0....|3.06|2.3723780287213376|
|[14.15,2.0,1

/usr/local/lib/python3.7/dist-packages/pyspark/sql/context.py:127: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


'Note': Prediction column show the prediction of tips for the worker